---
layout: post
title:  "Generative Adversarial Network"
date:   2023-03-22 10:14:54 +0700
categories: jekyll update
---

# Introduction

As the name suggests, GAN holds an adversarial relationship between two networks: a generator and a discriminator. The generator is a generative model that learns the data distribution of a population. Note that it doesn't learn directly, but via the prediction / classification / feedback of the discriminator. The discriminative model's job is to tell whether a sample comes from the data distribution or generated by the generator. A real world scenario for this framework is one in which the criminal tries to forge fake money and the police would need to tell fake from real money. In game theoretical framework, this is a two person zero sum game whose solution would be derived through minimax strategy: one tries to maximize his utility and the other tries to min it. There is a global solution in which the forgery looks the same as the real images/money (the generator learns the distribution), and the discriminator cannot tell the difference any more (the estimation of a sample to be fake is one half - effectively a random choice).

## Training

Initially, we input the correctly labeled images both from the generator and the data distribution to the discriminator, so that the discriminitor learns to predict with backpropagation. Training the generator is a bit more complicated since we need the feedback from the discriminator. So, we freeze the discriminator network, and use it as an agent of feedback. For the part of the generator, we input a random vector (from the space of all possibile vector for images - called the laten space) for the generator to output a fake image. This fake image is put through the discriminitor for classification, but we set the label for the fake image to be 1 (real). This is to maximize the mistake of the discriminator. The predicted result from the discriminator is put back to the generator so that the generator learns. This is roughly supervised learning. We use the cross entropy cost function between the discriminiator's $$ \hat{y} $$ and y true. In tensorflow, for the fully connected version of the GAN, we use ReLU activation, He normal initialization, binary cross entropy loss and RMSProp optimizer.

### Fully Connected GAN

In [ ]:

Dense = tf.keras.layers.Dense
generator = tf.keras.Sequential([
    Dense(100, activation="relu", kernel_initializer="he_normal"),
    Dense(150, activation="relu", kernel_initializer="he_normal"),
    Dense(28 * 28, activation="sigmoid"),
    tf.keras.layers.Reshape([28, 28])
])
discriminator = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    Dense(150, activation="relu", kernel_initializer="he_normal"),
    Dense(100, activation="relu", kernel_initializer="he_normal"),
    Dense(1, activation="sigmoid")
])
gan = tf.keras.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")



### Deep CNN GAN

In [ ]:
generator = tf.keras.Sequential([
    tf.keras.layers.Dense(7 * 7 * 128),
    tf.keras.layers.Reshape([7, 7, 128]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2,
                                    padding="same", activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2DTranspose(1, kernel_size=5, strides=2,
                                    padding="same", activation="tanh"),
])
discriminator = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, kernel_size=5, strides=2, padding="same",
                        activation=tf.keras.layers.LeakyReLU(0.2)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(128, kernel_size=5, strides=2, padding="same",
                        activation=tf.keras.layers.LeakyReLU(0.2)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
gan = tf.keras.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")



### Deeper CNN GAN

In [ ]:
def build_discriminator(depth=64, p=0.4): # Define inputs
    image = Input((img_w,img_h,1))
    # Convolutional layers
    conv1 = Conv2D(depth*1, 5, strides=2,
                   padding='same', activation='relu')(image)
    conv1 = Dropout(p)(conv1)
    conv2 = Conv2D(depth*2, 5, strides=2,
               padding='same', activation='relu')(conv1)
    conv2 = Dropout(p)(conv2)
    conv3 = Conv2D(depth*4, 5, strides=2,
                   padding='same', activation='relu')(conv2)
    conv3 = Dropout(p)(conv3)
    conv4 = Conv2D(depth*8, 5, strides=1,
               padding='same', activation='relu')(conv3)
    conv4 = Flatten()(Dropout(p)(conv4))
    # Output layer
    
    prediction = Dense(1, activation='sigmoid')(conv4)
    # Model definition
    model = Model(inputs=image, outputs=prediction) 
    return model
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
                      optimizer=RMSprop(lr=0.0008, decay=6e-8,
                                        clipvalue=1.0),
                      metrics=['accuracy'])

def build_generator(latent_dim=z_dimensions,
                      depth=64, p=0.4):
    noise = Input((latent_dim,))
    # First dense layer
    dense1 = Dense(7*7*depth)(noise)
    dense1 = BatchNormalization(momentum=0.9)(dense1)
    dense1 = Activation(activation='relu')(dense1)
    dense1 = Reshape((7,7,depth))(dense1)
    dense1 = Dropout(p)(dense1)
    # De-Convolutional layers
    conv1 = UpSampling2D()(dense1)
    conv1 = Conv2DTranspose(int(depth/2),
                            kernel_size=5, padding='same', activation=None,)(conv1) 
    conv1 = BatchNormalization(momentum=0.9)(conv1)
    conv1 = Activation(activation='relu')(conv1)
    conv2 = UpSampling2D()(conv1)
    conv2 = Conv2DTranspose(int(depth/4),
                            kernel_size=5, padding='same',
                            activation=None,)(conv2) 
    conv2 = BatchNormalization(momentum=0.9)(conv2)
    conv2 = Activation(activation='relu')(conv2)
    conv3 = Conv2DTranspose(int(depth/8),
                            kernel_size=5, padding='same',
                            activation=None,)(conv2) 
    conv3 = BatchNormalization(momentum=0.9)(conv3)
    conv3 = Activation(activation='relu')(conv3)
    # Output layer
    image = Conv2D(1, kernel_size=5, padding='same',
                   activation='sigmoid')(conv3)
    # Model definition
    model = Model(inputs=noise, outputs=image) 
    return model

generator = build_generator()

img = generator(z) 
discriminator.trainable = False 
pred = discriminator(img) 
adversarial_model = Model(z, pred)

adversarial_model.compile(loss='binary_crossentropy',
                          optimizer=RMSprop(lr=0.0004, decay=3e-8,
                                            clipvalue=1.0),
                          metrics=['accuracy'])



## CIFAR-10